# k/v RDDs

## Trabajando con pares llave/valor    (key/value, k/v)

### - De forma natural, bastantes entidades se pueden representar como llaves:
    - p. ej., tiempo de eventos (timestamp), id de cliente, etc.
### - En Python, usamos tuplas para formar pares k/v
```python
>>> ("taquitos", 4), ("burritos", 2)
```



## Pair RDDs

### A aquellos RDDs que contienen pares k/v los llamamos  **Pair RDDs** y están compuestos de tuplas:
```python
pairs = sc.parallelize([("a", 2), ("b", 4), ("c", 6)])
```

### Spark ofrece operaciones especiales en **Pair RDDs**:
    - reduceByKey, sortByKey, joins
    - Son funciones que solo operan en tuplas
### Los Pair RDDs también soportan las mismas funciones que RDDs regulares

In [ ]:
from pyspark import SparkContext
sc = SparkContext(master = "local[*]")

---

## reduceByKey()

### - Ejecuta multiples operaciones de reducción (reduce ops); una por cada llave en el dataset
### - Cada reducción combina valores que tienen la misma llave
### - reduceByKey() no es una acción como reduce()
### - Regresa un RDD nuevo (y no un valor)



```python

>> rdd = sc.parallelize([("a", 2), ("b", 4), ("b", 6)])

>> rdd.reduceByKey(lambda x, y: x + y)
```

In [ ]:
rdd = sc.parallelize([("a", 2), ("b", 4), ("b", 6)])

rdd.reduceByKey(lambda x, y: x + y).collect()

## sortByKey()

### - Regresa un RDD ordenado según la llave

### - `sortByKey(ascending = True/False, keyFunc = lambda... )`
    - Con ordenes ascendentes/descendientes
    - Se puede proveer funciones de comparación
    
    
    
```python

>> rdd = sc.parallelize([('c', 2), ('a', 3), ('b', 1)])

>> rdd.sortByKey()

>> rdd.sortByKey(False)
```

In [ ]:
rdd = sc.parallelize([('c', 2), ('a', 3), ('b', 1)])

rdd.sortByKey().collect()

In [ ]:
rdd.sortByKey(False).collect()

## groupByKey()

```python

>> rdd = sc.parallelize([("a", 2), ("b", 4), ("b", 6)])

>> rdd.groupByKey()

>>> [('b', [4, 6]), ('a', [2])]
```

### - 💅🏼🔥 groupByKey() puede causar una sobre-carga de datasets a ordenarse entre workers! 🙈🚒
### - [Avoid GroupByKey: Databricks Spark Knowledge Base](https://databricks.gitbooks.io/databricks-spark-knowledge-base/content/best_practices/prefer_reducebykey_over_groupbykey.html)

In [ ]:
rdd = sc.parallelize([("a", 2), ("b", 4), ("b", 6)])

rdd.groupByKey().mapValues(list).collect()

## mapValues()

### - Retorna un nuevo RDD al aplicar una función a cada valor del RDD original


```python
>> rdd = sc.parallelize([("a", 1), ("b", 2), ("b", 3)])

>> rdd.mapValues(lambda x: x * 2)
```

In [ ]:
rdd = sc.parallelize([("a", 1), ("b", 2), ("b", 3)])

rdd.mapValues(lambda x: x * 2).collect()

## Pair RDD actions

### - Todas las acciones de RDDs básicos están disponibles



## countByKey()

### - Conteo de número de incidencias para cada llave

```python
>> rdd = sc.parallelize([('a', "doc1"), ('b', 'doc1'), ('b', 'doc2')])

>> rdd.countByKey()
```


In [ ]:
rdd = sc.parallelize([('a', "doc1"), ('b', 'doc1'), ('b', 'doc2')])

rdd.countByKey()

## collectAsMap()

### - Búsqueda mediante un diccionario

```python
>> rdd = sc.parallelize([("a", 2), ("b", 4), ("c", 6)])

>> dd = rdd.collectAsMap()
```

In [ ]:
rdd = sc.parallelize([("a", 2), ("b", 4), ("c", 6)])

dd = rdd.collectAsMap()

dd

In [ ]:
dd['b']

In [ ]:
type(dd)

# Partitions

### - Cada RDD siempre se separa en un número de particiones

### - El grado de paralelismo es determinado por el número de particiones
- [Parallelism vs Concurrency](https://wiki.haskell.org/Parallelism_vs._Concurrency)

### - rdd.getNumberOfPartitions()

### RDD Partitions
![RDD Partitions](https://github.com/israelzuniga/dlatam-bigdata-workshop/blob/master/notebooks/img/rdd_partitions.png?raw=true)

## Cómo definir el número de particiones?

### - ```sc.textFile(path, minPartitions=None)```


### - repartition() puede ser usada para un nuevo dataset con otro número de particiones

### - Útil cuando se conoce previamente la cantidad de llaves

In [ ]:
x = sc.textFile('README.md', 6)

In [ ]:
x.repartition()

## Data partitioning


### - El particionamiento determina como se agrupan las llaves en el mismo nodo

### - Útil para operaciones como reduceByKey, joins, cogroup, etc...

### - Reducen la comunicación de red entre nodos 💰

## No olvidar:

### - Algunas operaciones no preservan el particionamiento (p. ej. map())


### - Se debe usar mapValues() o flatMapValues() en vez de map(), siempre que no se necesite cambiar la llave

### - Es recomendable usar partitionBy() antes de operaciones tipo join y reduce en datasets extensos

# Joins


## - pySpark soporta cuatro tipos de uniones en RDDs:

###  ```join(), leftOuterJoin(), rightOuterJoin(), fullOuterJoin()```

## - Su uso es más común en Pair RDDs

### - P. Ej.: Combinar un dataset de clientes con otro dataset de compras

## join()

### join() funciona de la misma manera que un "inner join" en SQL

### Solo las llaves presentes en ambos RDDs se preservan

### Retorna todos los pares de elementos como tuplas

In [ ]:
x = sc.parallelize([("a", 1), ("b", 4)])

y = sc.parallelize([("a", 2), ("a", 3)])

In [ ]:
x.join(y)

## leftOuterJoin

### ``` x.leftOuterJoin(y) ```

In [ ]:
x = sc.parallelize([('a', 1), ('b', 4)])

y = sc.parallelize([('a', 2)])

In [ ]:
x.leftOuterJoin(y)

## rightOuterJoin

### ``` y.leftOuterJoin(x) ```

In [ ]:
x = sc.parallelize([('a', 1), ('b', 4)])

y = sc.parallelize([('a', 2)])

In [ ]:
y.rightOuterJoin(x)

## fullOuterJoin()

### ``` x.fullOuterJoin(y) ```

In [ ]:
x = sc.parallelize([('a', 1), ('b', 4)])

y = sc.parallelize([('a', 2), ('c', 8)])

In [ ]:
x.fullOuterJoin(y)

---



---



---



---



# Review:

- Pair RDDs

- Partitions

- Joins

In [ ]:
sc.stop()